In [0]:
from pyspark.sql.functions import *

order_items = spark.read.json('/public/retail_db_json/order_items')
orders = spark.read.json('/public/retail_db_json/orders')

In [0]:
# AGGREGATIONS

# Basic total aggregation
orders.select(count('*').alias('count'), sum('order_id').alias('sum')).show()

# Aggregation by specific column
order_items.groupBy('order_item_id').sum('order_item_quantity').show(n=10)

# Aggregation using .agg() method - far more flexible
order_items\
    .groupBy('order_item_id')\
    .agg(sum('order_item_quantity'), sum('order_item_subtotal').alias('order_revenue'), min('order_item_quantity'))\
    .show(n=5)

In [0]:
# Basic aggregation of whole dataset (Total Aggregation)

orders.select(count('*').alias('count'), sum('order_id').alias('sum')).show()

+-----+----------+
|count|       sum|
+-----+----------+
|68883|2372468286|
+-----+----------+



In [0]:
# .count() method can be used directly on a dataset

orders.count()

68883

In [0]:
# .groupBy method creates GroupedData datatype.
# It allows us to use aggregate by specific groups.

type(order_items.groupBy())

pyspark.sql.group.GroupedData

In [0]:
# .groupBy() without an argument lets us aggregate the whole dataset (that can be aggregated with used function)
# For example: min() will not return the value for a string column

order_items.groupBy().min().show()

+------------------+------------------------+--------------------------+-----------------------------+------------------------+------------------------+
|min(order_item_id)|min(order_item_order_id)|min(order_item_product_id)|min(order_item_product_price)|min(order_item_quantity)|min(order_item_subtotal)|
+------------------+------------------------+--------------------------+-----------------------------+------------------------+------------------------+
|                 1|                       1|                        19|                         9.99|                       1|                    9.99|
+------------------+------------------------+--------------------------+-----------------------------+------------------------+------------------------+



In [0]:
# .groupBy() with an argument lets us aggregate the dataset by specific columns
# .sum(*cols) will return the sum of all values for the given columns

order_items.groupBy('order_item_id').sum('order_item_quantity').show(n=10)

+-------------+------------------------+
|order_item_id|sum(order_item_quantity)|
+-------------+------------------------+
|           26|                       1|
|           29|                       1|
|          474|                       3|
|          964|                       3|
|         1677|                       2|
|         1697|                       1|
|         1806|                       2|
|         1950|                       1|
|         2040|                       5|
|         2214|                       4|
+-------------+------------------------+
only showing top 10 rows



In [0]:
# But this syntax is quite limited - we can use only one aggr function

order_items \
    .groupBy('order_item_id') \
    .sum('order_item_quantity') \
    .min('order_item_quantity') \
    .show()

---------------------------------------------------------------------------
PySparkAttributeError                     Traceback (most recent call last)
File <command-2567374494928662>, line 6
      1 # But this syntax is quite limited - we can use only one aggr function
      3 order_items \
      4     .groupBy('order_item_id') \
      5     .sum('order_item_quantity') \
----> 6     .min('order_item_quantity') \
      7     .show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3753, in DataFrame.__getattr__(self, name)
   3720 """Returns the :class:`Column` denoted by ``name``.
   3721 
   3722 .. versionadde

In [0]:
# And editing aggr column is also difficult

order_items\
    .groupBy('order_item_id')\
    .sum('order_item_quantity', 'order_item_subtotal')\
    .toDF('order_item_order_id', 'order_quantity', 'order_revenue')\
    .withColumn('order_revenue', round('order_revenue', 2))\
    .show(n=10)

+-------------------+--------------+-------------+
|order_item_order_id|order_quantity|order_revenue|
+-------------------+--------------+-------------+
|                 26|             1|       129.99|
|                 29|             1|        59.99|
|                474|             3|       119.97|
|                964|             3|       119.97|
|               1677|             2|        99.96|
|               1697|             1|       399.98|
|               1806|             2|       119.98|
|               1950|             1|        49.98|
|               2040|             5|       499.95|
|               2214|             4|       239.96|
+-------------------+--------------+-------------+
only showing top 10 rows



In [0]:
# A better way is to use .agg() method
# Then we can use multiple aggr functions and edit the aggr columns on the fly

order_items\
    .groupBy('order_item_id')\
    .agg(sum('order_item_quantity'), sum('order_item_subtotal').alias('order_revenue'), min('order_item_quantity'))\
    .show(n=5)

+-------------+------------------------+-------------+------------------------+
|order_item_id|sum(order_item_quantity)|order_revenue|min(order_item_quantity)|
+-------------+------------------------+-------------+------------------------+
|           26|                       1|       129.99|                       1|
|           29|                       1|        59.99|                       1|
|          474|                       3|       119.97|                       3|
|          964|                       3|       119.97|                       3|
|         1677|                       2|        99.96|                       2|
+-------------+------------------------+-------------+------------------------+
only showing top 5 rows

